In [1]:
def analyze(filename):
    print()
    print("Analyzing: {}".format(filename))
    df = pd.read_csv(filename,encoding='utf-8')
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])
            
def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result)) 



In [2]:
import tensorflow.contrib.learn as skflow
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

path = "./data/"

filename_read = os.path.join(path,"auto-mpg.csv")
analyze(filename_read)




Analyzing: ./data/auto-mpg.csv
398 rows
** mpg:129 (32%)
** cylinders:[4:51.26%,8:25.88%,6:21.11%,3:1.01%,5:0.75%]
** displacement:[97.0:5.28%,98.0:4.52%,350.0:4.52%,250.0:4.27%,318.0:4.27%,140.0:4.02%,400.0:3.27%,225.0:3.27%,91.0:3.02%,232.0:2.76%,121.0:2.76%,302.0:2.76%,151.0:2.51%,120.0:2.26%,231.0:2.01%,200.0:2.01%,90.0:2.01%,85.0:2.01%,351.0:2.01%,304.0:1.76%,122.0:1.76%,105.0:1.76%,156.0:1.51%,79.0:1.51%,119.0:1.51%,108.0:1.26%,107.0:1.26%,89.0:1.26%,258.0:1.26%,135.0:1.26%,360.0:1.01%,86.0:1.01%,116.0:1.01%,112.0:1.01%,305.0:1.01%,134.0:1.01%,455.0:0.75%,307.0:0.75%,429.0:0.75%,173.0:0.75%,198.0:0.75%,168.0:0.75%,113.0:0.75%,260.0:0.75%,146.0:0.75%,70.0:0.75%,383.0:0.5%,71.0:0.5%,163.0:0.5%,262.0:0.5%,141.0:0.5%,199.0:0.5%,440.0:0.5%,104.0:0.25%,390.0:0.25%,454.0:0.25%,340.0:0.25%,110.0:0.25%,267.0:0.25%,88.0:0.25%,111.0:0.25%,144.0:0.25%,181.0:0.25%,145.0:0.25%,100.0:0.25%,81.0:0.25%,183.0:0.25%,131.0:0.25%,78.0:0.25%,80.0:0.25%,130.0:0.25%,72.0:0.25%,101.0:0.25%,115.0:0.25%,1

In [3]:
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name]-df[name].mean())>=(sd*df[name].std()))]
    df.drop(drop_rows,axis=0,inplace=True)
    
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)    
    
def encode_numeric_zscore(df,name,mean=None,sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name]-mean)/sd
    
    

In [4]:
import tensorflow.contrib.learn as skflow
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

path = "./data/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

missing_median(df, 'horsepower')
df.drop('name',1,inplace=True)
encode_numeric_zscore(df, 'horsepower')
encode_numeric_zscore(df, 'weight')
encode_numeric_zscore(df, 'cylinders')
encode_numeric_zscore(df, 'displacement')
encode_numeric_zscore(df, 'acceleration')

print("Length before MPG outliers dropped: {}".format(len(df)))
remove_outliers(df,'mpg',2)
print("Length after MPG outliers dropped: {}".format(len(df)))

df



Length before MPG outliers dropped: 398
Length after MPG outliers dropped: 388


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,18.0,1.496308,1.089233,0.672271,0.630077,-1.293870,70,1
1,15.0,1.496308,1.501624,1.587959,0.853259,-1.475181,70,1
2,18.0,1.496308,1.194728,1.195522,0.549778,-1.656492,70,1
3,16.0,1.496308,1.060461,1.195522,0.546236,-1.293870,70,1
4,17.0,1.496308,1.041280,0.933897,0.565130,-1.837804,70,1
5,15.0,1.496308,2.259274,2.451322,1.618455,-2.019115,70,1
6,14.0,1.496308,2.499036,3.026898,1.633806,-2.381737,70,1
7,14.0,1.496308,2.364769,2.896085,1.584210,-2.563048,70,1
8,14.0,1.496308,2.508627,3.157710,1.717647,-2.019115,70,1
9,15.0,1.496308,1.885244,2.242022,1.038654,-2.563048,70,1


In [5]:
import requests

address = "1301 Administration Ave., Fargo, ND 58102"

response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address='+address)

resp_json_payload = response.json()

print(resp_json_payload['results'][0]['geometry']['location'])



{'lat': 46.8921907, 'lng': -96.8013713}


In [6]:
from math import sin, cos, sqrt, atan2, radians

def distance_lat_lng(lat1,lng1,lat2,lng2):
    R = 6373.0

    lat1 = radians(lat1)
    lng1 = radians(lng1)
    lat2 = radians(lat2)
    lng2 = radians(lng2)

    dlng = lng2 - lng1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlng / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

def lookup_lat_lng(address):
    response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address='+address)
    json = response.json()
    if len(json['results']) == 0:
        print("Can't find: {}".format(address))
        return 0,0
    map = json['results'][0]['geometry']['location']
    return map['lat'],map['lng']


import requests

address1 = "1301 Administration Ave., Fargo, ND 58102" 
address2 = "1250 14th Street, Denver, CO 80204"

lat1, lng1 = lookup_lat_lng(address1)
lat2, lng2 = lookup_lat_lng(address2)

print("Distance from NDSU to UoC: {} km".format(
        distance_lat_lng(lat1,lng1,lat2,lng2)))



Distance from NDSU to UoC: 1034.113606563466 km


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'This is the first document.',
    'This is the second document.',
    'And here is the third one.',
    'Is this the first or second document?']

vectorizer = CountVectorizer(min_df=1)

vectorizer.fit(corpus)

print("Mapping")
print(vectorizer.vocabulary_)

print()
print("Encoded")
x = vectorizer.transform(corpus)
print(x.toarray())



Mapping
{'this': 10, 'is': 4, 'the': 8, 'first': 2, 'document': 1, 'second': 7, 'and': 0, 'here': 3, 'third': 9, 'one': 5, 'or': 6}

Encoded
[[0 1 1 0 1 0 0 0 1 0 1]
 [0 1 0 0 1 0 0 1 1 0 1]
 [1 0 0 1 1 1 0 0 1 1 0]
 [0 1 1 0 1 0 1 1 1 0 1]]


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

path = "./data/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

corpus = df['name']

vectorizer = CountVectorizer(min_df=1)

vectorizer.fit(corpus)

print("Mapping")
print(vectorizer.vocabulary_)

print()
print("Encoded")
x = vectorizer.transform(corpus)
print(x.toarray())

print(len(vectorizer.vocabulary_))

bag_cols = [0] * len(vectorizer.vocabulary_)
for i,key in enumerate(vectorizer.vocabulary_):
    bag_cols[i] = key

    

Mapping
{'chevrolet': 94, 'chevelle': 91, 'malibu': 185, 'buick': 75, 'skylark': 251, '320': 37, 'plymouth': 220, 'satellite': 243, 'amc': 62, 'rebel': 228, 'sst': 257, 'ford': 143, 'torino': 274, 'galaxie': 147, '500': 44, 'impala': 167, 'fury': 145, 'iii': 166, 'pontiac': 221, 'catalina': 84, 'ambassador': 61, 'dpl': 131, 'dodge': 129, 'challenger': 88, 'se': 245, 'cuda': 115, '340': 39, 'monte': 202, 'carlo': 83, 'estate': 136, 'wagon': 293, 'sw': 268, 'toyota': 276, 'corona': 107, 'mark': 188, 'ii': 165, 'duster': 132, 'hornet': 164, 'maverick': 191, 'datsun': 123, 'pl510': 219, 'volkswagen': 290, '1131': 4, 'deluxe': 125, 'sedan': 247, 'peugeot': 215, '504': 47, 'audi': 67, '100': 1, 'ls': 179, 'saab': 239, '99e': 56, 'bmw': 73, '2002': 22, 'gremlin': 154, 'f250': 138, 'chevy': 95, 'c20': 77, 'd200': 120, 'hi': 161, '1200d': 7, 'vega': 285, '2300': 26, 'pinto': 218, 'custom': 116, 'matador': 190, 'brougham': 74, 'monaco': 200, 'country': 110, 'squire': 256, 'safari': 240, 'sportab

In [9]:
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor

y = df['mpg'].as_matrix()

forest = RandomForestRegressor(n_estimators=50,
                              random_state=0, verbose = True)
forest.fit(x, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

print("Feature ranking:")

for f in range(x.shape[1]):
    print("{}. {} ({})".format(f + 1, bag_cols[f], importances[indices[f]]))

    

Feature ranking:
1. chevrolet (0.069858186873746)
2. chevelle (0.0657962678049988)
3. malibu (0.051008883870636394)
4. buick (0.04581221475540706)
5. skylark (0.04198230223754908)
6. 320 (0.040399764416811264)
7. plymouth (0.032608231867797544)
8. satellite (0.028083396361630455)
9. amc (0.025365169053742708)
10. rebel (0.020822620576517337)
11. sst (0.02065610924024198)
12. ford (0.019690892181255654)
13. torino (0.019012669846068458)
14. galaxie (0.01750032394359091)
15. 500 (0.014938020006637028)
16. impala (0.014369710481955338)
17. fury (0.01252127394359739)
18. iii (0.010751429685292812)
19. pontiac (0.010702428667434061)
20. catalina (0.010542674460379622)
21. ambassador (0.01025593869126505)
22. dpl (0.00986759162389823)
23. dodge (0.009133861291584486)
24. challenger (0.008772300799934196)
25. se (0.008600514953250088)
26. cuda (0.007821470049179373)
27. 340 (0.007294524170328526)
28. monte (0.007132502583116468)
29. carlo (0.007099310584072058)
30. estate (0.00707868872699017

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.2s finished
